## Key words
### 경계값, threshold, 승산비, OR, Logit, LogisticRegression, roc_auc_curve, predict_proba, accuracy_score, f1_score, precision_score, recall_score

### 로지스틱 회귀분석 특징
- `이항` 로지스틱 회귀 분석은 종속변수가 0과 1이며 베르누이 분포를 따를 경우 사용
 - 종속변수의 특징에 따라 순서형 로지스틱, 명목형 로지스틱도 있음
 - 여기서는 이항 로지스틱 회귀 분석을 알아볼 것임
- 모델의 산출값은 각 데이터가 1이 될 확률이며 이진분류를 위해서 경계값이 필요
- 모델 평가를 위해 각종 분류 관련 지표 및 AUC(Area Under Curve) 활용

### 승산비(OR, Odds Ratio)
- 특정 독립변수를 제외한 나머지 값을 고정하고 해당 독립변수가 1증가 시 변화하는 승산(odds)의 비
- Numpy의 exp함수 사용

### statsmodels - Logit() [ols 함수처럼 모델 생성 함수]
- 로지스틱 회귀분석을 실시하는 statsmodels의 함수
- endog, exog 인자에 각각 종속변수와 독립변수를 할당
- 산출 모델 객체의 params 어트리뷰트에 모델의 계수 저장
- 산출 모델 객체의 predict() 메서드로 예측값을 생산하며 이는 종속변수가 1이 될 확률 값

In [2]:
import pandas as pd
import numpy as np
from statsmodels.api import Logit
# 나중에 배울것 미리 부름

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [3]:
df = pd.read_csv("iris.csv")
df.head(2)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa


In [4]:
df["Species"].unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [5]:
# 파생변수 만들기
df["is_setosa"] = (df["Species"] == "setosa") + 0 # 종속변수로 사용할것임
df.head(2)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,is_setosa
0,5.1,3.5,1.4,0.2,setosa,1
1,4.9,3.0,1.4,0.2,setosa,1


In [6]:
# model = Logit(endog = df["is_setosa"],
#              exog = df.iloc[:, :4]).fit()
# model # 독립변수의 값이 너무 똑같으면 오류발생 -> 비슷한 컬럼인덱스 빼줘야함

In [7]:
model = Logit(endog = df["is_setosa"],
              exog = df.iloc[:, :2]).fit()
model 

Optimization terminated successfully.
         Current function value: 0.036374
         Iterations 11


In [38]:
dir(model)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cache',
 '_data_attr',
 '_data_in_cache',
 '_get_endog_name',
 '_get_robustcov_results',
 '_use_t',
 'aic',
 'bic',
 'bse',
 'conf_int',
 'cov_kwds',
 'cov_params',
 'cov_type',
 'df_model',
 'df_resid',
 'f_test',
 'fittedvalues',
 'get_margeff',
 'initialize',
 'k_constant',
 'llf',
 'llnull',
 'llr',
 'llr_pvalue',
 'load',
 'mle_retvals',
 'mle_settings',
 'model',
 'nobs',
 'normalized_cov_params',
 'params',
 'pred_table',
 'predict',
 'prsquared',
 'pvalues',
 'remove_data',
 'resid_dev',
 'resid_generalized',
 'resid_pearson',
 'resid_response',
 'save',
 'scale',
 'set_null_options',
 'summary'

In [41]:
print(model.summary2)

<bound method DiscreteResults.summary2 of <statsmodels.discrete.discrete_model.LogitResults object at 0x0000017FD008AF40>>


In [8]:
model.params # 종속변수를 1로 예측하기 위한 독립변수의 계수들

Sepal.Length    -7.529945
Sepal.Width     13.130734
dtype: float64

In [9]:
# 필요한 값 tab 눌러서 찾아넣으면됨 model.[tab]
model.pvalues # 각각의 변수에 대한 p-value 값

Sepal.Length    0.000828
Sepal.Width     0.000989
dtype: float64

In [10]:
pred = model.predict(df.iloc[:3, :2]) # 1번~3번이 setosa일 확률이 99, 92, 99%라는 얘기
pred

0    0.999477
1    0.923824
2    0.998678
dtype: float64

In [11]:
(pred > 0.5) + 0 # threshold=0.5, 이런식으로 수치형으로 바꿔서 후처리하기도함, 이진분류

0    1
1    1
2    1
dtype: int32

---
### sklearn - LogisticRegression() - [ols(), Logit()처럼 모델 생성하는 함수]
- statsmodels의 Logit()보다 좀더 옵션이 많음
 - 최적값 찾기엔 이게 좋고, 통계적으로는 statsmodels가 더좋음
- 로지스틱 회귀분석을 실시하는 sklearn 함수
- fit_intercept, solver 인자로 절편 적합 여부 및 최적화 알고리즘 설정 가능
- random_state 인자에 자연수를 할당하여 결과 고정 가능
- fit() 메서드에 독립변수 및 종속변수 할당
- 산출 모델 객체의 coef_ 어트리뷰트에 모델의 계수 저장
- 산출 모델 객체의 predict_proba() 메서드로 예측값을 생산하며 두 번째 열이 종속변수가 1이 될 확률값

In [12]:
from sklearn.linear_model import LogisticRegression

In [13]:
df.head(2)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,is_setosa
0,5.1,3.5,1.4,0.2,setosa,1
1,4.9,3.0,1.4,0.2,setosa,1


In [14]:
model = LogisticRegression(random_state = 123).fit(X = df.iloc[:, :2], y = df["is_setosa"])
model

LogisticRegression(random_state=123)

In [15]:
model.coef_

array([[-3.38829757,  3.1645277 ]])

In [16]:
model.intercept_

array([8.32330389])

In [17]:
model.predict_proba(df.iloc[:3, :2]) # 2열이 is_setosa가 1이 될확률

array([[0.10727976, 0.89272024],
       [0.22895365, 0.77104635],
       [0.07413821, 0.92586179]])

In [18]:
pred = model.predict_proba(df.iloc[:3, :2])
pred = pred[:, 1]
pred

array([0.89272024, 0.77104635, 0.92586179])

In [19]:
(pred > 0.5) + 0 # threshold = 0.5, 이진분류

array([1, 1, 1])

### sklearn - roc_auc_score()
- AUC(Area Under Curve)[그래프 아래면적값]를 산출하는 sklearn의 함수
- y_true, y_score 인자에 각각 종속변수와 예측 확률값 할당
 - y_true는 실제 종속변수 학습할 때 사용한 것
 - y_score는 예측한 값

In [20]:
pred = model.predict_proba(df.iloc[:, :2])
pred = pred[:, 1]
pred[:10]

array([0.89272024, 0.77104635, 0.92586179, 0.92738323, 0.94126096,
       0.91436651, 0.97058885, 0.89484454, 0.93034007, 0.82210603])

In [21]:
from sklearn.metrics import roc_auc_score

In [22]:
roc_auc_score(y_true= df["is_setosa"],
              y_score = pred) # 최대값이 1, 최소값이 0.5에 가까움

0.9999999999999999

- 1에 가까우므로 선형회귀 R-squared(결정계수) 1에 가까운 것 만큼 좋은 거임

---
## 오분류 관련 함수
### sklearn - accuracy_score()
- 분류모델의 정확도를 산출하는 sklearn의 함수
- y_pred와 y_true에 각각 예측 분류 결과와 실제 값을 할당

### sklearn - f1_score()
- 분류모델의 f1 값을 산출하는 sklearn의 함수
- y_pred와 y_true에 각각 예측 분류 결과와 실제 값을 할당

### sklearn - preicison_score()
- 분류모델의 정밀도(precision)를 산출하는 sklearn의 함수
- y_pred와 y_true에 각각 예측 분류 결과와 실제 값을 할당

### sklearn - recall_score()
- 분류모델의 재현율(recall)를 산출하는 sklearn의 함수
- y_pred와 y_true에 각각 예측 분류 결과와 실제 값을 할당

In [23]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [24]:
df["is_setosa"].head()

0    1
1    1
2    1
3    1
4    1
Name: is_setosa, dtype: int32

In [25]:
accuracy_score(y_true= df["is_setosa"], # y_true에 test값이면 test값, 원래값이면 원래값 할당
               y_pred = (pred > 0.5) + 0) # y_pred에 예측 분류 결과 할당
# 1이 나오므로 완벽하다는 뜻

1.0

In [26]:
accuracy_score(y_true= df["is_setosa"],
               y_pred = (pred > 0.8) + 0) # 1이 나오므로 완벽하다는 뜻

0.9466666666666667

In [27]:
accuracy_score(y_true= df["is_setosa"],
               y_pred = (pred > 0.9) + 0) # 1이 나오므로 완벽하다는 뜻

0.8333333333333334

In [28]:
precision_score(y_true= df["is_setosa"],
               y_pred = (pred > 0.5) + 0)

1.0

In [29]:
recall_score(y_true= df["is_setosa"],
               y_pred = (pred > 0.5) + 0)

1.0

In [30]:
f1_score(y_true= df["is_setosa"],
               y_pred = (pred > 0.5) + 0)

1.0

### 1. 독립변수를 혈압, 혈당, BMI, 인슐린으로 하고 종속변수를 당뇨 여부로 할 경우 분류 정확도는 얼마인가?
- diabetes.csv
- statsmodels 함수 사용
- 데이터는 학습: 평가 = 8:2 로 분리 후 계산
- Seed는 123
- threshold 언급이 없다면 0.5로 하고 실시

In [31]:
df = pd.read_csv("diabetes.csv")
df.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0


In [1]:
from sklearn.model_selection import train_test_split

In [33]:
df_train, df_test = train_test_split(df, train_size=0.8, random_state=123) 

In [34]:
model = Logit(endog = df_train["Outcome"],
              exog = df_train.iloc[:, [1, 2, 4, 5]]).fit()
model

Optimization terminated successfully.
         Current function value: 0.626579
         Iterations 5


In [35]:
df_train.iloc[:, [1, 2, 4, 5]].head(2)

,Glucose,BloodPressure,Insulin,BMI
318,115,66,140,38.1
313,113,50,85,29.5


In [36]:
pred = model.predict(df_test.iloc[:, [1, 2, 4, 5]])
pred

236    0.462956
395    0.507051
36     0.359735
210    0.314389
483    0.219876
         ...   
650    0.398274
579    0.561881
119    0.281386
593    0.386267
310    0.280089
Length: 154, dtype: float64

In [37]:
accuracy_score(y_true = df["Outcome"],
               y_pred = (pred > 0.5) + 0)

ValueError: Found input variables with inconsistent numbers of samples: [768, 154]

정답

In [ ]:
# 나머지는 다맞음

In [ ]:
accuracy_score(y_true = df_test["Outcome"],
               y_pred = (pred > 0.5) + 0)

### 2. 독립변수를 혈당, BMI, 나이로 하고 종속변수를 당뇨 여부로 할 경우 나이의 승산비는 얼마인가?
- diabetes.csv
- statsmodels 함수 사용

정답

In [42]:
import numpy as np

In [43]:
df = pd.read_csv("diabetes.csv")
df.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0


In [44]:
model = Logit(endog = df["Outcome"],
              exog = df.loc[:, ["Glucose", "BMI", "Age"]]).fit()
model.params

Optimization terminated successfully.
         Current function value: 0.656276
         Iterations 4


Glucose    0.009368
BMI       -0.035639
Age       -0.012898
dtype: float64

In [45]:
np.exp(model.params) # 각각의 odds ratio가 나옴

Glucose    1.009412
BMI        0.964989
Age        0.987184
dtype: float64

### 3. 독립변수를 혈당, BMI, 나이로하고 종속변수를 당뇨여부로 할 경우 모델의 AUC는 얼마인가?
- diabetes.csv
- statsmodels 함수 사용

In [ ]:
df = pd.read_csv("diabetes.csv")
df.head(2)

In [ ]:
model = Logit(endog = df["Outcome"],
              exog = df.loc[:, ["Glucose", "BMI", "Age"]]).fit()
model

In [ ]:
pred = model.predict(df.loc[:, ["Glucose", "BMI", "Age"]])
pred[:10]

In [ ]:
roc_auc_score(y_true= df["Outcome"],
              y_score = pred) # 최대값이 1, 최소값이 0.5에 가까움